# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key




### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
cities_df = pd.read_csv('output_data/cities.csv')
cities_df = cities_df.dropna()
cities_df.head()

,Unnamed: 0,City,Temperature,Humidity,Cloudiness,Latitude,Longitude,Wind Speed
0,0,Santa cruz de la palma,17.64,77.0,0.0,28.68,-17.76,4.10
2,2,Kapaa,25.85,65.0,40.0,22.08,-159.32,8.20
3,3,Ushuaia,6.00,81.0,75.0,-54.80,-68.30,11.80
4,4,Faanui,27.98,70.0,37.0,-16.48,-151.75,1.03
5,5,Butaritari,28.65,80.0,52.0,3.07,172.79,7.96


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
#configure gmaps with figure layout
gmaps.configure(api_key = g_key)
figure_layout = {
    'width': '1000 px', 
    'height': '1000 px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure()

locations = cities_df[['Latitude', 'Longitude']].astype(float) # locations df with coordinates as floats
humidity = cities_df['Humidity'].astype(float) #humidity df with humidity as floats

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=1) #configure heat layer
fig.add_layer(heat_layer) #add layer to figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [38]:
# Only pick cities where temp is between 18 and 25 celcius, wind speed below 10 mph, and less than 20% cloudiness
picky_df = cities_df.loc[
    (cities_df['Temperature'] > 18) & (cities_df['Temperature'] < 25) &
    (cities_df['Wind Speed'] < 10) &
    (cities_df['Cloudiness'] < 20),
     :]
picky_df['Hotel Name'] = ''
picky_df['Hotel Lat'] = ''
picky_df['Hotel Lon'] = ''


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [32]:
# values are inserted into picky_df to keep things cohesive
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    'rankby': 'distance',
    'type': 'hotel',
    'key': g_key
}

for index, row in picky_df.iterrows():
    params['location'] = f'{row["Latitude"]},{row["Longitude"]}' #build location parameter from picky_df
    response = requests.get(base_url, params = params).json() #call the googlemaps nearby search api
    results = response['results']
    #add data for each hotel to picky_df
    picky_df.loc[index, 'Hotel Name'] = results[0]['name']
    picky_df.loc[index, 'Hotel Lat'] = results[0]['geometry']['location']['lat']
    picky_df.loc[index, 'Hotel Lon'] = results[0]['geometry']['location']['lng']


picky_df


,Unnamed: 0,City,Temperature,Humidity,Cloudiness,Latitude,Longitude,Wind Speed,Hotel Name,Hotel Lat,Hotel Lon
36,36,Barranca,19.93,84.0,2.0,-10.75,-77.77,2.49,Yuri De Los Santos,-10.7513,-77.7709
57,57,Sao filipe,24.19,67.0,0.0,14.90,-24.50,3.60,Pensão Ngozi Nova Cidade,14.9004,-24.4985
92,92,Airai,20.32,47.0,3.0,-8.93,125.41,1.07,Raw Material Atsabe,-8.92162,125.41
105,105,Beipiao,19.11,51.0,0.0,41.79,120.78,4.16,People's Park,41.7929,120.775
119,119,Vila velha,23.46,88.0,16.0,-20.33,-40.29,1.00,Atelier do Maçom,-20.3301,-40.2903
...,...,...,...,...,...,...,...,...,...,...,...
1274,1274,Caravelas,23.49,83.0,15.0,-17.71,-39.25,2.22,Mateus' S/A,-17.7157,-39.2459
1336,1336,Lompoc,21.50,38.0,1.0,34.64,-120.46,4.10,Qineti Q Mission Solutions,34.64,-120.46
1422,1422,Port hedland,19.00,25.0,0.0,-20.32,118.57,3.60,Port Hedland Harbour - Port Facilities,-20.3162,118.569
1464,1464,Pascagoula,24.51,31.0,1.0,30.37,-88.56,2.60,SBM LLC,30.37,-88.56


In [34]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in picky_df.iterrows()]
locations = picky_df[["Hotel Lat", "Hotel Lon"]]

In [39]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))